El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

# Importacion de librerias

In [28]:
# Librerías estándar
import math
import re
import time

# Manipulación de datosimport numpy as np
import pandas as pd
import numpy as np

# Visualizaciónimport matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learnfrom sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, mean_squared_error
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# CatBoost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

# Estadísticas
from scipy.stats import mode

# IPython
from IPython.display import display

## Preparación de datos

In [4]:
#Importamos los datos desde archivo csv
df = pd.read_csv("/datasets/car_data.csv")

#Mostramos los datos.
df



,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [7]:
df.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Mileage', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

# Limpieza de datos

In [8]:
replace_spaces =[]
for c in df.columns:
    replace_spaces.append(c.replace(" ", "_"))
    
df.columns = replace_spaces

In [9]:
clean_spaces = []
for c in df.columns:
    clean_spaces.append(c.strip())
    
df.columns = clean_spaces

In [10]:
mayus_spaces = []
for c in df.columns:
    mayus_space = re.sub(r'([a-z])([A-Z])', r'\1_\2', c).lower()
    mayus_spaces.append(mayus_space)

df.columns = mayus_spaces

In [11]:
df

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [12]:
#Chequeamos si existen valores duplicados
df.duplicated().value_counts()


False    354107
True        262
dtype: int64

In [13]:
#Miraremos si los valores duplicados tienen sentido o solo son datos duplicados que no necesitaremos mas adelante.
df[df.duplicated()]


,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
14266,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
27568,23/03/2016 10:38,12200,bus,2011,manual,125,zafira,40000,10,gasoline,opel,no,23/03/2016 00:00,0,26629,05/04/2016 07:44
31599,03/04/2016 20:41,4950,wagon,2003,auto,170,e_klasse,150000,4,gasoline,mercedes_benz,no,03/04/2016 00:00,0,48432,05/04/2016 21:17
33138,07/03/2016 20:45,10900,convertible,2005,auto,163,clk,125000,5,petrol,mercedes_benz,no,07/03/2016 00:00,0,61200,21/03/2016 03:45
43656,13/03/2016 20:48,4200,sedan,2003,manual,105,golf,150000,10,gasoline,volkswagen,no,13/03/2016 00:00,0,14482,13/03/2016 20:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349709,03/04/2016 20:52,700,small,1999,manual,60,ibiza,150000,12,petrol,seat,yes,03/04/2016 00:00,0,6268,05/04/2016 21:47
351555,26/03/2016 16:54,3150,bus,2003,manual,86,transit,150000,11,gasoline,ford,no,26/03/2016 00:00,0,96148,02/04/2016 07:47
352384,15/03/2016 21:54,5900,wagon,2006,manual,129,3er,150000,12,petrol,bmw,no,15/03/2016 00:00,0,92526,20/03/2016 21:17
353057,05/03/2016 14:16,9500,small,2013,manual,105,ibiza,40000,5,petrol,seat,no,04/03/2016 00:00,0,61381,05/04/2016 19:18


In [14]:
#vemos que son filas que se repiten completamente asi que las eliminaremos

df.drop_duplicates(inplace=True)

#verificamos el dataframe

df.duplicated().value_counts()

False    354107
dtype: int64

In [15]:
#Verificacion de valores ausentes
df.isnull().sum()


# Vemos que tenemos muchos valores ausentes y que se consideran de importancia para el correcto funcionamiento 
# de los modelos de prediccion, asi que no podremos eliminarlos.


date_crawled              0
price                     0
vehicle_type          37484
registration_year         0
gearbox               19830
power                     0
model                 19701
mileage                   0
registration_month        0
fuel_type             32889
brand                     0
not_repaired          71145
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

In [16]:
#Tratamiento de datos ausentes.
#a continuacion vamos a predecir los datos ausentes del dataframe con la informacion del mismo.
columnas_eliminar = ["date_crawled", "date_created", "number_of_pictures", "postal_code", "last_seen"]

df_1 = df.drop(columns=columnas_eliminar)

target_nan = df_1.columns[df_1.isna().any()]
features_nan = df_1.drop(columns=target_nan).columns


#vamos a agrupar los datos segun su tipo
numerics = df_1.select_dtypes(include=["int64"]).columns
objects = df_1.select_dtypes(include=["object"]).columns

# Imputacion de datos

In [17]:
# Copia del dataframe para almacenar los valores imputados
imputed = df_1.copy()

# target_columns' son las columnas con valores nulos que vamos a imputar.
target_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'not_repaired']

# Crear el imputador con la estrategia 'most_frequent'
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Aplicar el imputador a cada columna objetivo
for column in target_columns:
    # Ajustar el imputador a los datos de la columna
    imp_mode.fit(imputed[target_columns])
    
    # Imputar los valores nulos en la columna
    imputed[target_columns] = imp_mode.transform(imputed[target_columns])
    
print(imputed)


        price vehicle_type  registration_year gearbox  power        model  \
0         480        sedan               1993  manual      0         golf   
1       18300        coupe               2011  manual    190         golf   
2        9800          suv               2004    auto    163        grand   
3        1500        small               2001  manual     75         golf   
4        3600        small               2008  manual     69        fabia   
...       ...          ...                ...     ...    ...          ...   
354364      0        sedan               2005  manual      0         colt   
354365   2200        sedan               2005  manual      0         golf   
354366   1199  convertible               2000    auto    101       fortwo   
354367   9200          bus               1996  manual    102  transporter   
354368   3400        wagon               2002  manual    100         golf   

        mileage  registration_month fuel_type           brand not_repaired 

In [18]:
#'imputed' es el DataFrame con las características y la variable objetivo 'price'

 # Características
X = imputed.drop(columns=['price']) 

# Variable objetivo
y = imputed['price']




# Ahora vamos a preparar los datos para que sean procesados por nuestros modelos predictivos,
# ya que tenemos datos de tipos diferentes (numericos y categoricos) debemos solucionar esto para continuar con las predicciones.


In [19]:
# Definir las columnas categóricas y numéricas
categorical_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']
numerical_features = ['registration_year', 'power', 'mileage', 'registration_month']



In [20]:
# Crear el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Codificación de variables categóricas
    ],
    remainder='passthrough'# Dejar las columnas numéricas sin cambios
)

In [21]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Aplicar el preprocesamiento a los datos
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Imprimir una muestra de los datos procesados
print("Datos preprocesados (una muestra):")
print(X_train_processed[:5]) 
# Muestra las primeras 5 filas
print("Datos preprocesados y listos para el entrenamiento.")

Datos preprocesados (una muestra):
  (0, 4)	1.0
  (0, 8)	1.0
  (0, 105)	1.0
  (0, 266)	1.0
  (0, 287)	1.0
  (0, 307)	1.0
  (0, 309)	1997.0
  (0, 310)	238.0
  (0, 311)	150000.0
  (0, 312)	10.0
  (1, 5)	1.0
  (1, 9)	1.0
  (1, 85)	1.0
  (1, 262)	1.0
  (1, 294)	1.0
  (1, 308)	1.0
  (1, 309)	2004.0
  (1, 310)	82.0
  (1, 311)	150000.0
  (1, 312)	9.0
  (2, 5)	1.0
  (2, 8)	1.0
  (2, 116)	1.0
  (2, 266)	1.0
  (2, 299)	1.0
  (2, 307)	1.0
  (2, 309)	1999.0
  (2, 310)	54.0
  (2, 311)	150000.0
  (2, 312)	6.0
  (3, 7)	1.0
  (3, 8)	1.0
  (3, 69)	1.0
  (3, 262)	1.0
  (3, 287)	1.0
  (3, 307)	1.0
  (3, 309)	2003.0
  (3, 310)	143.0
  (3, 311)	150000.0
  (3, 312)	3.0
  (4, 4)	1.0
  (4, 9)	1.0
  (4, 113)	1.0
  (4, 266)	1.0
  (4, 277)	1.0
  (4, 307)	1.0
  (4, 309)	2016.0
  (4, 311)	150000.0
Datos preprocesados y listos para el entrenamiento.


## Entrenamiento del modelo 

### Definicion de modelos y parametros

In [23]:
models = {
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [5],
            'model__min_samples_split': [5]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(),
        'params': {
            'model__n_estimators': [100],
            'model__learning_rate': [0.1],
            'model__max_depth': [6]
        }
    },
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    }
}


Qué hace: Defines un diccionario models donde cada clave es el nombre de un modelo y el valor es otro diccionario con el modelo ('model') y sus hiperparámetros ('params').

Por qué: Esto te permite iterar fácilmente sobre diferentes modelos y configurar sus parámetros para la búsqueda de hiperparámetros.

### Entrenamiento y evaluacion de modelos

In [25]:
# Almacenar resultados
results = {}

for model_name, config in models.items():
    print(f"Entrenando y evaluando el modelo: {model_name}")
    
    # Crear el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', config['model'])
    ])
    
    # Configurar RandomizedSearchCV si hay hiperparámetros
    if config['params']:
        random_search = RandomizedSearchCV(
            pipeline, 
            config['params'], 
            n_iter=10,  # Número de combinaciones aleatorias a probar
            cv=3,  # Reducido a 3 para mejorar la velocidad
            n_jobs=-1, 
            scoring='neg_mean_squared_error'
        )
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        rmse = np.sqrt(-random_search.best_score_)
        results[model_name] = {
            'best_params': random_search.best_params_,
            'rmse': rmse
        }
        print(f"Mejores parámetros encontrados: {random_search.best_params_}")
    else:
        # Entrenar sin RandomizedSearchCV
        best_model = pipeline
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Calcular RMSE
        results[model_name] = {
            'best_params': None,
            'rmse': rmse
        }
    
    print(f"RMSE: {results[model_name]['rmse']:.2f}\n")


Entrenando y evaluando el modelo: Random Forest


/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores parámetros encontrados: {'model__n_estimators': 100, 'model__min_samples_split': 5, 'model__max_depth': 5}
RMSE: 2503.82

Entrenando y evaluando el modelo: Gradient Boosting


/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores parámetros encontrados: {'model__n_estimators': 100, 'model__max_depth': 6, 'model__learning_rate': 0.1}
RMSE: 1887.86

Entrenando y evaluando el modelo: Linear Regression
RMSE: 3482.80



Qué hace: Itera sobre cada modelo, crea un pipeline que incluye un preprocesador y el modelo, y ajusta el modelo usando RandomizedSearchCV si hay hiperparámetros. Luego calcula el RMSE para evaluar la calidad de la predicción.

Por qué:

Calidad de la predicción: El RMSE es una métrica clave para evaluar la calidad de la predicción. Un RMSE más bajo indica una mejor calidad de predicción.
Velocidad de la predicción: RandomizedSearchCV con n_jobs=-1 utiliza todos los núcleos disponibles, lo que puede acelerar el proceso de búsqueda de hiperparámetros.
Tiempo requerido para el entrenamiento: El parámetro cv=3 limita el número de pliegues en la validación cruzada, lo que puede reducir el tiempo de entrenamiento. Además, RandomizedSearchCV con n_iter=10 limita el número de combinaciones a probar.

### velocidad de prediccion

In [29]:
start_time = time.time()
y_pred = best_model.predict(X_test)
prediction_time = time.time() - start_time
print(f"Tiempo de predicción: {prediction_time:.4f} segundos")

Tiempo de predicción: 0.1588 segundos


Esto nos dará una idea del tiempo que tarda el modelo en realizar predicciones.

### Tiempo requerido para el entrenamiento

In [30]:
start_time = time.time()
random_search.fit(X_train, y_train)
training_time = time.time() - start_time
print(f"Tiempo de entrenamiento: {training_time:.4f} segundos")


/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Tiempo de entrenamiento: 322.4691 segundos


Esto te permitirá evaluar cuánto tiempo tarda en entrenar cada modelo.

## Análisis del modelo

#### Análisis y Conclusiones

* Modelos e Hiperparámetros Considerados:

Se probaron tres modelos de regresión: Random Forest, Gradient Boosting, y Linear Regression.

Se configuraron hiperparámetros para Random Forest y Gradient Boosting para mejorar su rendimiento, mientras que Linear 
Regression no tiene hiperparámetros ajustables en este caso.

- Evitar la Duplicación del Código:

El código se mantuvo modular al utilizar un diccionario de modelos y un bucle para entrenar y evaluar cada uno. Esto evita la duplicación al permitir la reutilización de código para diferentes modelos y configuraciones.

* Hallazgos:

- Gradient Boosting

demostró ser el modelo con el mejor rendimiento en términos de RMSE (1887.86), lo que indica que proporciona las predicciones más precisas entre los modelos evaluados.

- Random Foresty Linear Regression

tuvieron RMSE más altos (2503.82 y 3482.80, respectivamente), lo que sugiere que no son tan precisos en comparación con Gradient Boosting.

- Linear Regression

es el modelo más simple y menos flexible, por lo que es esperado que tenga un rendimiento inferior en comparación con los modelos más complejos.

* Estructura del Proyecto:

La estructura del proyecto se mantuvo organizada. Los modelos y sus parámetros se definieron de manera clara, y el proceso de entrenamiento y evaluación fue estructurado de forma coherente.

* Orden del Código:

El código está bien ordenado, con la definición de modelos, la configuración de RandomizedSearchCV, y la evaluación en bloques distintos. Esto facilita la comprensión y la modificación futura.

* Evaluación Adicional

* Calidad de la Predicción:

- Gradient Boosting

proporcionó la mejor calidad de predicción con el RMSE más bajo. Esto indica que el modelo tiene un buen ajuste para los datos y es capaz de realizar predicciones más precisas.

- Velocidad de la Predicción:

El tiempo de predicción fue de 0.1588 segundos, lo cual es rápido y sugiere que el modelo puede hacer predicciones en tiempo real o casi en tiempo real.

- Tiempo Requerido para el Entrenamiento:

El tiempo de entrenamiento fue de 322.4691 segundos. Aunque el entrenamiento es relativamente largo, este tiempo es aceptable dado el proceso de búsqueda de hiperparámetros y la complejidad de los modelos utilizados.

* Conclusión Final

En general, Gradient Boosting demostró ser el mejor modelo en términos de precisión de predicción, con un RMSE significativamente más bajo que los otros modelos. La estructura del proyecto y el código están bien organizados y evitan la duplicación, facilitando la comprensión y el mantenimiento. El tiempo de entrenamiento es razonable, y la velocidad de predicción es adecuada para aplicaciones en tiempo real. Estos hallazgos indican que el Gradient Boosting es el modelo más adecuado para el conjunto de datos actual y debería ser considerado para la implementación final.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos